# Testing

In [ ]:
from helpers import load_session, get_all_driver_features, merge_driver_features_with_session, load_or_build_profiles, build_circuit_profile_df

import pandas as pd
import numpy as np

In [ ]:
YEAR          = 2024
EVENT_NAME    = "Spanish Grand Prix"     # exactly as FastF1 knows it
SESSION_LABEL = "FP2"

session_data  = load_session(YEAR, EVENT_NAME, SESSION_LABEL)

if session_data["status"] != "ok":      # FastF1 returned an error and no
    raise RuntimeError(session_data["reason"])  # fallback OpenF1 yet

session       = session_data["session"]  # FastF1.Session object

In [ ]:
driver_df = get_all_driver_features(
    session,
    year=YEAR,
    session_name=SESSION_LABEL,
    throttle_ratio_min=0.40,
    throttle_ratio_max=0.85,
    braking_drop_kmh=30
)
print(driver_df.head(3))

In [ ]:
driver_df

In [ ]:
df_profiles, df_skipped = load_or_build_profiles(
    cache_path="data/circuit_profiles.csv",
    start_year=2024,
    end_year=2025
)

In [ ]:
df_profiles.loc[df_profiles.event=='Spanish Grand Prix']

In [ ]:
merge_driver_features_with_session(driver_df, df_profiles)

In [24]:
import fastf1 as ff1
s = ff1.get_event_schedule(2025, backend="fastf1").head()

In [33]:
def _session_list(
    event_format: str,
    *,
    testing_labels: str = "fastf1"
) -> list[str]:
    """
    Return the session labels we expect telemetry for, given the
    EventFormat string that comes from `ff1.get_event_schedule`.

    Parameters
    ----------
    event_format : str
        Ergast/FastF1 field – e.g. "conventional", "sprint_qualifying",
        "sprint_shootout", "testing".
    testing_labels : {"fastf1", "f1timing"}
        Which naming convention to use for *testing* events:

        - "fastf1"   → FP1 / FP2 / FP3  
        - "f1timing" → Day 1 / Day 2 / Day 3

    Returns
    -------
    list[str]
        Ordered list of session strings to iterate over.
    """
    fmt = (event_format or "").lower()

    # 1) Normal race weekend
    if fmt == "conventional":
        return ["FP1", "FP2", "FP3", "Q", "R"]

    # 2) Sprint weekend (covers both "sprint_qualifying" & "sprint_shootout")
    if fmt.startswith("sprint"):
        return ["FP1", "SQ", "S", "Q", "R"]

    # 3) Pre-season or in-season testing
    if fmt == "testing":
        if testing_labels == "f1timing":
            return ["Day 1", "Day 2", "Day 3"]
        # default: FastF1 naming
        return ["FP1", "FP2", "FP3"]

    # 4) Fallback – be conservative
    return ["FP1", "FP2", "FP3", "Q", "R"]

In [34]:
_session_list('testing')

['FP1', 'FP2', 'FP3']

# Work in progress

1. quali pace
2. race pace
3. Pirelli tire info
4. when to query the data?
    - the time of the race is not consistent
    - on sprint race weekends the quali takes place on Friday
    - US/Asia/Australia races happen at different times

In [8]:
def create_pirelli_tyre_info(location, traction, evolution, lateral_load, abrasion, braking, grip, tyre_stress, downforce, year):
    return pd.DataFrame([{
        'location': location,
        'traction': traction,
        'evolution': evolution,
        'lateral_load': lateral_load,
        'abrasion': abrasion,
        'braking': braking,
        'grip': grip,
        'tyre_stress': tyre_stress,
        'downforce': downforce,
        'year': year
        }])


In [9]:
bahrain_pirelli = create_pirelli_tyre_info('Sakhir', 4, 4, 3, 5, 4, 3, 3, 3)
jeddah_pirelli = create_pirelli_tyre_info('Jeddah', 2, 3, 4, 2, 2, 3, 3, 2)
melbourne_pirelli = create_pirelli_tyre_info('Melbourne', 2, 4, 3, 2, 2, 3, 3, 3)
baku_pirelli = create_pirelli_tyre_info('Baku', 5, 5, 1, 1, 4, 1, 3, 1)
miami_pirelli = create_pirelli_tyre_info('Miami',3,5,3,2,3,3,3,2)
monaco_pirelli = create_pirelli_tyre_info('Monaco', 5,5,1,1,2,1,1,5)
barcelona_pirelli = create_pirelli_tyre_info('Barcelona', 3,3,5,4,3,3,5,4) #2023
barcelona_pirelli = create_pirelli_tyre_info('Barcelona', 3,3,4,4,3,3,4,4) #2022
montreal_pirelli = create_pirelli_tyre_info('Montreal', 5,5,1,2,5,1,3,1)


TypeError: create_pirelli_tyre_info() missing 1 required positional argument: 'year'

In [5]:
pirelli = pd.concat([
    bahrain_pirelli,
    jeddah_pirelli,
    melbourne_pirelli,
    baku_pirelli,
    miami_pirelli ,
    monaco_pirelli ,
    barcelona_pirelli,
    montreal_pirelli ,
    ], 
    ignore_index=True
    )

In [6]:
pirelli

,Location,traction,evolution,lateral_load,abrasion,braking,grip,tyre_stress,downforce
0,Sakhir,4,4,3,5,4,3,3,3
1,Jeddah,2,3,4,2,2,3,3,2
2,Melbourne,2,4,3,2,2,3,3,3
3,Baku,5,5,1,1,4,1,3,1
4,Miami,3,5,3,2,3,3,3,2
5,Monaco,5,5,1,1,2,1,1,5
6,Barcelona,3,3,4,4,3,3,4,4
7,Montreal,5,5,1,2,5,1,3,1


In [ ]:

spielberg_pirelli = create_pirelli_tyre_info('Spielberg', 2,3,5)
silverstone_pirelli = create_pirelli_tyre_info('Silverstone', 4,4,10)
budapest_pirelli = create_pirelli_tyre_info('Budapest', 3,7,4)
spa_pirelli = create_pirelli_tyre_info('Spa', 3,8,8)
imola_pirelli = create_pirelli_tyre_info('Imola', 5,3,11)
abu_dhabi_pirelli = create_pirelli_tyre_info('Abu Dhabi',3,6,7)
las_vegas_pirelli = create_pirelli_tyre_info('Las Vegas', 7,4,6)
mexico_pirelli = create_pirelli_tyre_info('Mexico City', 7,6,4)
sao_paulo_pirelli = create_pirelli_tyre_info('Sao Paulo', 1,6,8)
suzuka_pirelli = create_pirelli_tyre_info('Suzuka', 3,4,11)
qatar_pirelli = create_pirelli_tyre_info('Qatar', 1,4,11)
austin_pirelli = create_pirelli_tyre_info('Austin', 7,5,8)
zandvoort_pirelli = create_pirelli_tyre_info('Zandvoort', 2,6,6)
monza_pirelli = create_pirelli_tyre_info('Monza', 2,5,4)
singapore_pirelli = create_pirelli_tyre_info('Singapore', 10,8,5) #2022
singapore_pirelli = create_pirelli_tyre_info('Singapore', 7,7,5) #2023